# Text Parser Testing document

This document is going to read the text file found in ../generated_data/Logs2020OCR_avepdf.com_horizontal.txt which was generated using the following website:


[avepdf.com](http://avepdf.com/)

In [ ]:
import WilliamstownData as wd

In [ ]:
#read in the data 
filename='../generated_data/Logs2019OCR_avepdf.com_horizontal.txt'
dict2019 = wd.parsefile(filename)
filename='../generated_data/Logs2020OCR_avepdf.com_horizontal.txt'
dict2020 = wd.parsefile(filename)


lines = dict2019[0]+dict2020[0]
calls = dict2019[1]+dict2020[1]
call_dicts = dict2019[2]+dict2020[2]

In [ ]:
#Review call parcing data
from ipywidgets import interact

def view_both(index=0):
    print("RAW CALL DATA\n")
    for line in calls[index]:
        print(line)
    print('========================================')
    print("PARSED DATA\n")
    for key in call_dicts[index]:
        print(f"{key}: {call_dicts[index][key]}")
    
target = interact(view_both, index=(0,len(calls)-1))

In [ ]:
#Exmaple Count number of STOPS by "Craig"
total_stops = 0 
tot = 0
total_race = {}
norace = 0
my_stops = []
for call in call_dicts:
    if 'call taker' in call:
        if "hammer" in call['call taker'].lower():
            tot += 1
#             print(call['callReason'])
            if "STOP" in call['reason']:
                my_stops.append(call)
                total_stops += 1
                if 'operator_race' in call:
                    race = call['operator_race'].lower()
                    if race in total_race:
                        total_race[race] = total_race[race]+1
                    else:
                        total_race[race] = 1
                else:
                    norace +=1

In [ ]:
total_stops

In [ ]:
tot

In [ ]:
total_race

In [ ]:
#Make a list of names
def name_counter(call_dicts):
    name_dict = {}

    for call in call_dicts:
        if 'call taker' in call:
            name = call['call taker']
            if name in name_dict:
                name_dict[name] = name_dict[name] + 1
            else:
                name_dict[name] = 1
    return name_dict

name_dict = name_counter(call_dicts)

In [ ]:
name_dict

In [ ]:
from fuzzywuzzy import fuzz, process
import pandas as pd
import numpy as np

df = pd.DataFrame()

names, count = [], []

for k,v in name_dict.items():
    names.append(k)
    count.append(v)
    
    
df['call taker'] = names
df['count'] = count

print(df.shape)
df.head(20)

In [ ]:
#Manullay determined after seeing repeats from above

top_names = ["TANIA HERNANDEZ",
"DAVID JENNINGS, D",
"SCOTT E MCGOWAN",
"BRAD SACCO",
"KALVIN DZIEDZIAK",
"JOHN J MCCONNELL JR",
"LAURIE TUPER",
"SHUAN N WILLIAM",
"KYLE J JOHNSON",
"DAVID R LEMIEUX",
"ANTHONY M DUPRAT",
"KEVIN P GARNER",
"MICHAEL J ZIEMBA Jr",
"SCOTT SKORUPSKI",
"CHRISTINE LEMOINE",
"WILLIAM C JENNINGS JR",
"MICHAEL J ZIEMBA Jr",
"CRAIG A EICHHAMMER",
"PAUL D THOMPSON",
"MICHAEL STRIZZI",
"LAURIE TOPER",
"ALL EQUIPMENT POLICE DEPARTMENT",
"BARB BRUCATO"]

In [ ]:
from fuzzywuzzy import fuzz, process

df['Fuzzy Names'] = ''

def fuzz_match():
    matched = []
    rank_list = ["CHIEF", "PATROL", "FATROL", "DISPATCHER", "SERGEANT", "LIEUTENANT", "'", "PATROLMMI", "PATROLMAN",
                'SERGEAN"T', "PATROU'1AN", "LIEUTRNANT", "SERGEAl<T", "DISJ?ATCHER", "SERGE~.NT", "?ATROL", "SERGBANT",
                "PAT~OL", "LIEUTENANI", ]
    for i, names in enumerate(df['call taker']): 
        name = names.split()

        rank = [rank for rank in rank_list if name[0] in rank]
        for r in rank:
            if r in name:
                name.remove(r)
            continue

        name = ' '.join(name)
            
        df.loc[i, 'Fuzzy Names'] = name

        highest = process.extractOne(name,top_names)
        match = highest[0]

        df.loc[i, 'Matched'] = match            
    return df
df = fuzz_match()

In [ ]:
df

In [ ]:
lookup = {}
for index, row in df.iterrows():
    lookup[row['call taker']] = row['Matched']
lookup

In [ ]:
for call in call_dicts:
    if 'call taker' in call:
        name = call['call taker']
        if name in lookup:
            call['call taker'] = lookup[name]

In [ ]:
name_dict = name_counter(call_dicts)

name_dict = {k: v for k, v in sorted(name_dict.items(), key=lambda item: item[1], reverse=True)}
name_dict


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(20,10))
plt.rcParams.update({'font.size':24})
plt.bar(list(name_dict.keys()), name_dict.values(), color='g')
plt.xticks(rotation=90);
plt.ylabel('Number of calls')
plt.xlabel('Officer Name')